# Temperature dependent elastic constants

## Background

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$U(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

### How to get $U$ or $\sigma$

- MD
- Quasi-Harmonic

## Tasks

- Get $a_0$ from potential
- Lattice parameter (as a function of T)
  - MD
    - NVT
    - NPT
  - QH
- Calculate $U$ or $\sigma$ for various $\varepsilon$
  - MD: Equilibriate and average with LAMMPS
  - QH: Get strains from Yuriy's tool and run phonopy
- Fit

## Teams

- MD: Erik, Han, (Raynol), Prabhath, Jan, Sriram
- QH: Raynol, (Sam), Bharathi, Ahmed, Haitham
- Fit & Yuriy: Sam
- Literature

# Implementation

* https://atomistics.readthedocs.io/en/latest/bulk_modulus_with_gpaw.html#elastic-matrix
* https://github.com/pyiron/atomistics/blob/main/tests/test_elastic_lammpslib_functional.py
* https://github.com/pyiron/pyiron_workflow_atomistics/blob/interstitials/pyiron_workflow_atomistics/dataclass_storage.py
* https://github.com/ligerzero-ai/pyiron_workflow_lammps/blob/main/pyiron_workflow_lammps/engine.py#L21

## Reference

We compare our values with the paper - [M. Krief, et. al., Physical Review E, 103, 063307, 2021](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.103.063307#s4)

Potential used: Copper [Mishin potential](https://www.ctcms.nist.gov/potentials/entry/2001--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-A-et-al--Cu-1/)

## Imports

In [1]:
from ase.build import bulk
from ase.atoms import Atoms

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import annotations

from atomistics.workflows.elastic.workflow import (
    get_tasks_for_elastic_matrix,
    analyse_results_for_elastic_matrix
)

from atomistics.calculators import (
    evaluate_with_lammpslib, 
    get_potential_by_name, 
    calc_molecular_dynamics_npt_with_lammpslib, 
    calc_molecular_dynamics_nvt_with_lammpslib
)

from atomistics.calculators.lammps.libcalculator import (
    calc_static_with_lammpslib, 
    calc_molecular_dynamics_langevin_with_lammpslib
)

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/__init__.py:41: UserWarning: calc_molecular_dynamics_phonons_with_lammpslib() is not available as the import of the module named 'dynaphopy' failed.
  raise_warning(module_list=lammps_phonon_functions, import_error=e)
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/__init__.py:94: UserWarning: calc_molecular_dynamics_phonons_with_lammpslib() is not available as the import of the module named 'dynaphopy' failed.
  raise_warning(module_list=lammps_phonon_functions, import_error=e)


In [3]:
from pyiron_base import Project, job

In [4]:
pr = Project("Finite_temp_convergence_3")

## Create bulk sample with a guessed lattice constant

In [5]:
unit_cell = bulk('Cu', 'fcc', a=3.6514, cubic=True) # 4 atoms

In [6]:
repeated_unit_cell = unit_cell.repeat(5) # 500 atoms

In [7]:
potential_name_str = "2001--Mishin-Y--Cu-1--LAMMPS--ipr1"

potential_df = get_potential_by_name(
    potential_name=potential_name_str
)

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


## 0K Relaxed Structure

In [8]:
def get_relaxed_structure_at_0K(
        structure: Atoms, 
        potential: str, 
        lmp_optimizer_kwargs : dict = None
        ) -> Atoms:
    
    df_pot_selected = get_potential_by_name(
        potential_name=potential
        )
    
    result_dict = evaluate_with_lammpslib(
        task_dict={"optimize_positions_and_volume": structure},
        potential_dataframe=df_pot_selected,
        lmp_optimizer_kwargs=lmp_optimizer_kwargs
        )
    
    structure_relaxed = result_dict['structure_with_optimized_positions_and_volume']

    return structure_relaxed

In [9]:
lmp_optimizer_kwargs={
    'min_style':'cg',
    'ionic_force_tolerance':1e-8,
    'pressure':np.zeros(6) # add anisotropy
}

In [10]:
relaxed_unit_cell = get_relaxed_structure_at_0K(
    unit_cell, # 4 atoms
    potential_name_str, 
    lmp_optimizer_kwargs
)

relaxed_unit_cell # 4 atoms

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: cmti001
--------------------------------------------------------------------------


Atoms(symbols='Cu4', pbc=True, cell=[3.6150000810785805, 3.6150000810785805, 3.6150000810785805])

## 0K Lattice Constant

In [11]:
def get_lattice_constant_at_0K(
        structure: Atoms, 
        potential: str, 
        lmp_optimizer_kwargs : dict = None
        ) -> float:

    structure_relaxed = get_relaxed_structure_at_0K(
        structure=structure, 
        potential=potential,
        lmp_optimizer_kwargs=lmp_optimizer_kwargs
    )
    
    a_0 = structure_relaxed.get_volume()**(1/3)

    return a_0 # Angstrom

In [12]:
a_0 = get_lattice_constant_at_0K(
    structure=unit_cell, 
    potential=potential_name_str,
    lmp_optimizer_kwargs=lmp_optimizer_kwargs)

a_0 # Angstrom

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


3.61500008107858

We get the same lattice constant at 0K as the reference paper!

## 0K Elastic Constants

In [13]:
def get_strain_tensor_cubic(
        structure : Atoms, 
        strain : float = 0.005
    ) -> dict:

    deformation_gradient_dict = {
        'C11': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0,      0,      0],
                                       [      0,      0,      0]]),
        'C12': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0, strain,      0], 
                                       [      0,      0,      0]]),
        'C44': np.eye(3,3) + np.array([[      0,      0,      0], 
                                       [      0,      0, strain], 
                                       [      0, strain,      0]])
    }

    return deformation_gradient_dict

In [14]:
def get_elastic_constants_from_stress_tensor(
        tensor_dict : dict, 
        strain : float
    ) -> list[float]:

    elastic_constants_list = []

    for constant_str, diff in tensor_dict.items():
        if constant_str == 'C11':
            constant = diff[0, 0] / strain
            elastic_constants_list.append(abs(constant))

        elif constant_str == 'C12':
            sigma33 = diff[2, 2]
            constant = (sigma33/ strain) / 2
            elastic_constants_list.append(abs(constant))

        elif constant_str == 'C44':
            sigma23 = diff[2, 1]
            constant = sigma23 / (2 * strain)
            elastic_constants_list.append(abs(constant))

    return elastic_constants_list

In [15]:
def get_stress_tensor_at_0K(
        structure : Atoms, 
        potential_dataframe : pd.DataFrame, 
        deformation_gradient : np.array
    ):
    
    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(
        strained_cell, 
        scale_atoms=True
    )
    
    relaxed_dict = calc_static_with_lammpslib(
        structure=structure, 
        potential_dataframe=potential_dataframe
    )
    strained_dict = calc_static_with_lammpslib(
        structure=structure_strained,
        potential_dataframe=potential_dataframe
    )

    relaxed_dict['stress_GPa'] = relaxed_dict['stress'] / 10**4
    strained_dict['stress_GPa'] = strained_dict['stress'] / 10**4

    stress_diff = strained_dict['stress_GPa'] - relaxed_dict['stress_GPa']
    
    return stress_diff, relaxed_dict, strained_dict

In [16]:
def calculate_elastic_constants_at_0K(
        structure : Atoms, 
        potential_name : str,
        strain : float = 0.005
    ):

    df_pot_selected = get_potential_by_name(
        potential_name=potential_name
    )

    deformation_gradient_dict = get_strain_tensor_cubic(
        structure=structure, 
        strain=strain
    )
    
    tensor_dict = {}
    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        diff, relaxed_dict, strained_dict = get_stress_tensor_at_0K(
            structure=structure,
            potential_dataframe=df_pot_selected,
            deformation_gradient=deformation_gradient
        )
        
        tensor_dict[constant_str] = diff
        tensor_dict[f'relaxed_dict_{constant_str}'] = relaxed_dict
        tensor_dict[f'strained_dict_{constant_str}'] = strained_dict
    
    elastic_constants_list = get_elastic_constants_from_stress_tensor(
        tensor_dict=tensor_dict, 
        strain=strain
    )

    return elastic_constants_list, tensor_dict

## Reference function to fit elastic constants (Jan + Yury)'s

Requires only `relaxed_unit_cell` and `potential_name_str` from previous cells

In [17]:
def fit_elastic_constants(
        structure: Atoms, 
        potential: str, 
        strains, 
        stresses=None, 
        energies=None):

    task_dict, sym_dict = get_tasks_for_elastic_matrix(
        structure=structure,
        eps_range=0.005,
        num_of_point=5,
        zero_strain_job_name="s_e_0",
        sqrt_eta=True
    )

    potential_df = get_potential_by_name(
        potential_name=potential
    )

    result_dict = evaluate_with_lammpslib(
        task_dict=task_dict,
        potential_dataframe=potential_df,
    )
    
    elastic_dict, sym_dict = analyse_results_for_elastic_matrix(
        output_dict=result_dict,
        sym_dict=sym_dict,
        fit_order=2,
        zero_strain_job_name="s_e_0",
    )

    return elastic_dict

In [18]:
elastic_dict = fit_elastic_constants(
    structure=relaxed_unit_cell,
    potential=potential_name_str,
    strains=None
)

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interactive_structure_setter(


In [19]:
elastic_dict['elastic_matrix']

array([[169.74837327, 123.55258251, 123.55258251,   0.        ,
          0.        ,   0.        ],
       [123.55258251, 169.74837327, 123.55258251,   0.        ,
          0.        ,   0.        ],
       [123.55258251, 123.55258251, 169.74837327,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  76.24914297,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         76.24914297,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  76.24914297]])

In [20]:
elastic_constants_list_reference = [
    elastic_dict['elastic_matrix'][0,0], 
    elastic_dict['elastic_matrix'][0,1], 
    elastic_dict['elastic_matrix'][3,3]
    ]

np.round(elastic_constants_list_reference, 1) # GPa

array([169.7, 123.6,  76.2])

In comparison with the [reference paper](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.103.063307#s4 "M. Krief, et. al., Physical Review E, 103, 063307, 2021"),

$C_{11}$=169.9GPa, $C_{12}$=122.6GPa, and $C_{44}$=76.2GPa

## Finite Temperature equlibiration
* First run NPT to relax volume
* Then equilibriate the cell by running NVT

In [21]:
def equilibriate_structure_at_finite_temperature(
        structure : Atoms, 
        potential : str, 
        temperature : float = 500,
        run : int = 100000,
        thermo : int = 100,
        seed : int = 4928459, 
        cell_scale_value : int = 5,
        thermostat : str = 'langevin'
    ) -> Atoms:

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )
    
    structure_repeated = structure.repeat(cell_scale_value)

    npt_dict = calc_molecular_dynamics_npt_with_lammpslib(
        structure=structure_repeated,
        potential_dataframe=df_pot_selected,
        Tstart=temperature,
        Tstop=temperature,
        run=run,
        thermo=thermo,
        seed=seed
    )
    
    npt_lattice_constant = (np.mean(npt_dict['volume'][20:]/len(structure_repeated))*len(structure))**(1/3)
    
    # FIXME: Make it for a generic element - something might be wrong here. Need to check error propagation
    # structure_npt = bulk('Cu', a=npt_lattice_constant, cubic=True)
    # structure_repeated_npt = structure_npt.repeat(cell_scale_value)
    
    structure_repeated_npt = structure.copy()
    structure_repeated_npt.set_cell(
        [[npt_lattice_constant,0,0], 
         [0,npt_lattice_constant,0], 
         [0,0,npt_lattice_constant]],
        scale_atoms = True
    )
    structure_repeated_npt = structure_repeated_npt.repeat(cell_scale_value)

    if thermostat == 'nose-hoover':
        nvt_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure_repeated_npt,
            potential_dataframe=df_pot_selected,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
        )
        
    elif thermostat == 'langevin':
        nvt_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure_repeated_npt,
            potential_dataframe=df_pot_selected,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
        )
        
    structure_repeated_nvt = structure_repeated_npt.copy()
    structure_repeated_nvt.set_cell(
        nvt_dict['cell'][-1]
    )
    structure_repeated_nvt.set_positions(
        nvt_dict['positions'][-1]
    )
    structure_repeated_nvt.set_velocities(
        nvt_dict['velocities'][-1]
    )

    return structure_repeated_nvt

## Temperature-dependent Elastic Constants

In [56]:
calc_molecular_dynamics_langevin_with_lammpslib?

Signature:
calc_molecular_dynamics_langevin_with_lammpslib(
    structure: 'Atoms',
    potential_dataframe: 'pandas.DataFrame',
    run: 'int' = 100,
    thermo: 'int' = 100,
    timestep: 'float' = 0.001,
    Tstart: 'float' = 100.0,
    Tstop: 'float' = 100,
    Tdamp: 'float' = 0.1,
    seed: 'int' = 4928459,
    dist: 'str' = 'gaussian',
    disable_initial_velocity: 'bool' = False,
    lmp=None,
    output_keys=('positions', 'cell', 'forces', 'temperature', 'energy_pot', 'energy_tot', 'pressure', 'velocities', 'volume'),
    **kwargs,
)
Docstring: <no docstring>
File:      /cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/libcalculator.py
Type:      function

In [57]:
def get_stress_tensor_at_finite_temperature(
        structure : Atoms, 
        potential_dataframe : pd.DataFrame, 
        deformation_gradient : np.array, 
        temperature : float,
        run : int, 
        thermo : int,
        seed : int,
        thermostat : str
    ):
    
    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(
        strained_cell, 
        scale_atoms=True
    )
    
    if thermostat == 'nose-hoover':
        relaxed_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
            # disable_initial_velocity=True
        )
        strained_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure_strained, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
            # disable_initial_velocity=True
        )
    
    elif thermostat == 'langevin':
        relaxed_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
            # disable_initial_velocity=True
        )
        strained_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure_strained, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
            # disable_initial_velocity=True
        )

    relaxed_dict['pressure_GPa'] = relaxed_dict['pressure'] / 10**4
    strained_dict['pressure_GPa'] = strained_dict['pressure'] / 10**4

    stress_diff = -np.mean(strained_dict['pressure_GPa'][20:] - relaxed_dict['pressure_GPa'][20:], axis=0)

    return stress_diff, relaxed_dict, strained_dict

In [58]:
def calculate_elastic_constants_at_finite_temperature(
        structure : Atoms, # change to unit cell
        cell_scale_value : int,
        potential_name : str, 
        temperature : float = 0, 
        strain : float = 0.005,
        run : int = 10000,
        thermo : int = 100, 
        seed : int = 42, 
        thermostat : str = 'langevin'
    ):

    df_pot_selected = get_potential_by_name(
        potential_name=potential_name
    )

    equilibriated_structure = equilibriate_structure_at_finite_temperature(
        structure=structure,
        potential=potential_name_str, 
        temperature=temperature, 
        seed=seed,
        cell_scale_value=cell_scale_value
    )
    
    deformation_gradient_dict = get_strain_tensor_cubic(
        structure=equilibriated_structure, 
        strain=strain
    )
    
    tensor_dict = {}
    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        diff, relaxed_dict, strained_dict = get_stress_tensor_at_finite_temperature(
            structure=equilibriated_structure, 
            potential_dataframe=df_pot_selected,
            deformation_gradient=deformation_gradient,
            temperature=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            thermostat=thermostat
        )
        
        tensor_dict[constant_str] = diff
        tensor_dict[f'relaxed_dict_{constant_str}'] = relaxed_dict
        tensor_dict[f'strained_dict_{constant_str}'] = strained_dict
    
    elastic_constants_list = get_elastic_constants_from_stress_tensor(
        tensor_dict=tensor_dict, 
        strain=strain
    )

    # results = [{
    #     "elastic_constants": elastic_constants_list,
    #     "tensor_dict": tensor_dict,
    #     # "input_args": locals()
    # }]

    # results_df = pd.DataFrame(results)
    # results_df.to_pickle(f"{pr.path}/results.pkl")

    return elastic_constants_list, tensor_dict

### Convergence tests

In [24]:
input_params_scale = {
    "cell_scale_value" : [3, 5, 7, 9],
    "run" : [10000],
    "temperature" : [500],
    "strain" : [0.0001],
    "seed": [1357],
    "thermostat" : ["langevin"]
}

In [25]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
1,30386264,finished,None,run_convergence_b24cc33aad3fb223df1edd5dae4d1cb1,/run_convergence_b24cc33aad3fb223df1edd5dae4d1cb1,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:11:23.665513,NaT,NaN,pchilaka@cmti001#2#cmti,PythonFunctionContainerJob,0.4,None,None
0,30386265,aborted,None,calculate_elastic_constants_at_finite_temperature_20f28325040f9ed4a8f9bb1090907812,/calculate_elastic_constants_at_finite_temperature_20f28325040f9ed4a8f9bb1090907812,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:11:33.487155,2025-10-15 17:12:19.370010,45.0,pchilaka@cmti099#80#cmti,PythonFunctionContainerJob,0.4,None,None
2,30386279,aborted,None,run_convergence_0feac5a3ac27813dff7d8074641b156f,/run_convergence_0feac5a3ac27813dff7d8074641b156f,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:15:17.645913,2025-10-15 17:15:22.342511,4.0,pchilaka@cmti001#2#cmti,PythonFunctionContainerJob,0.4,None,None


In [26]:
from itertools import product
keys = input_params_scale.keys()
values = input_params_scale.values()

for combo in product(*values):
    params = dict(zip(keys, combo))
    print(params)

{'cell_scale_value': 3, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 5, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 7, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 9, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}


In [34]:
input_params_scale = {
    "cell_scale_value" : [4],
    "run" : [10000],
    "temperature" : [500],
    "strain" : [0.0001],
    "seed": [1357],
    "thermostat" : ["langevin"]
}

In [ ]:
# def run_convergence(structure, potential_name_str, input_params, project):

#     from itertools import product
#     from pyiron_base import job
    
#     keys = input_params.keys()
#     values = input_params.values()

#     for combo in product(*values):
#         params = dict(zip(keys, combo))
#         print(params)

#         conv_job = job(calculate_elastic_constants_at_finite_temperature)
#         conv_out = conv_job(
#             structure = structure,
#             potential_name = potential_name_str,
#             pyiron_project = project,
#             **params
#         )

#         conv_out.server.queue = "cmti"
#         conv_out.server.cores = 80
#         conv_out.server.run_time = 21600

#         conv_future = conv_out.pull()

In [59]:
def run_convergence(structure, potential_name_str, input_params):

    from itertools import product
    from pyiron_base import job
    
    keys = input_params.keys()
    values = input_params.values()

    for combo in product(*values):
        params = dict(zip(keys, combo))
        print(params)

        elastic_list, tensor_dict = calculate_elastic_constants_at_finite_temperature(
            structure = structure,
            potential_name = potential_name_str,
            **params
        )

    return elastic_list, tensor_dict

In [60]:
elastic_list, tensor_dict = run_convergence(relaxed_unit_cell, potential_name_str, input_params_scale)

{'cell_scale_value': 4, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}


/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interact

In [61]:
elastic_list

[155.97140466021006, 333.15571147056687, 10.592895237158505]

In [55]:
elastic_list, tensor_dict = calculate_elastic_constants_at_finite_temperature(
    structure=relaxed_unit_cell,
    cell_scale_value=4,
    potential_name=potential_name_str,
    temperature=500,
    strain=0.0001,
    seed=1357,
    thermostat='langevin'
)

/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/cmmc/ptmp/pchilaka/Packages/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


Exception: ERROR: Illegal fix langevin command (src/fix_langevin.cpp:59)

In [36]:
convergence_job = job(run_convergence)

In [37]:
convergence_out = convergence_job(
    structure=relaxed_unit_cell,
    potential_name_str=potential_name_str,
    input_params = input_params_scale,
    project = pr,
    pyiron_project = pr
)

In [38]:
# convergence_out.server.queue = 'cmti'
# convergence_out.server.cores = 2
# convergence_out.server.run_time = 3600

In [39]:
convergence_out

In [40]:
convergence_future = convergence_out.pull()

The job run_convergence_f5b5749628530b6cdbb8f6ac15713727 was saved and received the ID: 30386307
{'cell_scale_value': 4, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
The job calculate_elastic_constants_at_finite_temperature_1b88a6914921284854001e258d3bfebe was saved and received the ID: 30386308
Queue system id:  18560072


* 1- outer - submitted
* 0- inner - submitted from 1
* 2- outer - submitted, meant to have inner running on same node
* 4- outer - ran locally
* 3- submitted from 4

In [45]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
1,30386264,finished,None,run_convergence_b24cc33aad3fb223df1edd5dae4d1cb1,/run_convergence_b24cc33aad3fb223df1edd5dae4d1cb1,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:11:23.665513,NaT,NaN,pchilaka@cmti001#2#cmti,PythonFunctionContainerJob,0.4,None,None
0,30386265,aborted,None,calculate_elastic_constants_at_finite_temperature_20f28325040f9ed4a8f9bb1090907812,/calculate_elastic_constants_at_finite_temperature_20f28325040f9ed4a8f9bb1090907812,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:11:33.487155,2025-10-15 17:12:19.370010,45.0,pchilaka@cmti099#80#cmti,PythonFunctionContainerJob,0.4,None,None
2,30386279,aborted,None,run_convergence_0feac5a3ac27813dff7d8074641b156f,/run_convergence_0feac5a3ac27813dff7d8074641b156f,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:15:17.645913,2025-10-15 17:15:22.342511,4.0,pchilaka@cmti001#2#cmti,PythonFunctionContainerJob,0.4,None,None
4,30386307,finished,None,run_convergence_f5b5749628530b6cdbb8f6ac15713727,/run_convergence_f5b5749628530b6cdbb8f6ac15713727,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:19:51.462979,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
3,30386308,aborted,None,calculate_elastic_constants_at_finite_temperature_1b88a6914921284854001e258d3bfebe,/calculate_elastic_constants_at_finite_temperature_1b88a6914921284854001e258d3bfebe,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_3/,2025-10-15 17:19:51.599265,2025-10-15 17:21:26.517270,94.0,pchilaka@cmti001#80#cmti,PythonFunctionContainerJob,0.4,None,None


## Some annoying testing

* 1 - submitted
* 3 - ran inside the submitted 1
* 0 - ran locally
* 2 - ran locally
* 4 - ran locally, submitted 5
* 5 - ran on compute node

In [ ]:
pr.job_table()

In [ ]:
tj = pr.load(30360556)

In [ ]:
tj.output

In [ ]:
for idx, row in pr.job_table().iterrows():
    pr.wait_for_job(row['job'], )

## Pyiron table

In [ ]:
def db_filter_function(job_table):
    return (job_table.status == "finished") & (job_table.hamilton == "PythonFunctionContainerJob")

def job_filter_function(job):
    return (job.status == "finished") & ("calculate" in job.name)

def get_input_structure(job):
    return job.project_hdf5['input']['data']['structure']

In [ ]:
table = pr.create.table("Convergence_pyiron_table", delete_existing_job=True)

# table.db_filter_function = db_filter_function
table.filter_function = job_filter_function

table.add["input_structure"] = get_input_structure

table.run(delete_existing_job=True)

In [ ]:
table.get_dataframe()